# Thruster angles

# Purpose
Check that thruster angles are post processed correctly, from the cos/sin

# Methodology
* Load data from one trip and see that the thruster angles makes sense.

# Setup

In [ ]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize
from src.data import get_dataset
from src.data import prepare_dataset
from src.data import trips
import scipy.integrate
import seaborn as sns
from src.visualization import visualize, animate

## Load trips:

In [ ]:
df = get_dataset.get(name='tycho_short_id')

In [ ]:
visualize.plot_trips(df=df, width=1400, height=800, zoom_start=14, color_key='trip_direction')

In [ ]:
trip = df.groupby('trip_no').get_group(129)
trip.head()

In [ ]:
visualize.plot_map(df=trip, width=1400, height=800, zoom_start=14, color_key='trip_direction')

In [ ]:
animate.widget(trip=trip)

In [ ]:
sin_keys = ['sin_pm%i' % n for n in range(1,5)]
cos_keys = ['cos_pm%i' % n for n in range(1,5)]
power_keys = ['power_em_thruster_%i' % n for n in range(1,5)]
columns = sin_keys + cos_keys + power_keys

In [ ]:
g = sns.PairGrid(trip[sin_keys])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)

In [ ]:
g = sns.PairGrid(trip[cos_keys])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)

In [ ]:
g = sns.PairGrid(trip[power_keys])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)

In [ ]:
df_mean =  df.groupby('trip_no').mean()

In [ ]:
g = sns.PairGrid(df_mean[power_keys + ['trip_direction']])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)

* It seems that thruster 1 & 2 are run together and that 3 & 4 are run togheter.
* thruster 3 & 4 have slightly higher power when ```trip_direction=1``` (*Helsingborg-Helsingør*), when these thruster are in the "aft" according to the positions:

In [ ]:
fig,ax=plt.subplots()
row = trip.iloc[0]
animate.plot_thrusters(ax=ax, row=row)
ax.set_aspect('equal')